In [1]:
import cv2
import numpy as np
import easyocr
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import torch
from PIL import Image
import json
import os
import re

C:\Users\User\anaconda3\envs\easyocr\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# # Initialize EasyOCR for Arabic and French
# reader = easyocr.Reader(['ar', 'en'])

# # Initialize TrOCR for handwritten text recognition
# processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-handwritten")
# model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-handwritten")


## Detection of bounding box using extraction of french and arabic sentences 

In [3]:
# # Initialize EasyOCR for Arabic and French
# reader = easyocr.Reader(['ar', 'en'])  # Changed 'en' to 'fr' since the form has French labels

# # Step 1: Preprocess the image
# def preprocess_image(image_path):
#     image = cv2.imread(image_path)
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
#     blur = cv2.GaussianBlur(thresh, (3, 3), 0)
#     cv2.imwrite("preprocessed_form.jpg", blur)
#     return blur, image

# # Step 2: Extract text using EasyOCR
# def extract_text_with_easyocr(image_path):
#     result = reader.readtext(image_path, detail=1)
#     extracted_data = []
#     for (bbox, text, confidence) in result:
#         center_x = (bbox[0][0] + bbox[2][0]) / 2
#         center_y = (bbox[0][1] + bbox[2][1]) / 2
#         extracted_data.append({
#             "text": text,
#             "bbox": bbox,
#             "center_x": center_x,
#             "center_y": center_y,
#             "confidence": confidence
#         })
#     return extracted_data

# # Step 3: Merge adjacent text regions
# def merge_adjacent_regions(extracted_data, y_threshold=5, x_threshold=10):
#     if not extracted_data:
#         return extracted_data
    
#     # Sort by center_y and then center_x to process regions from left to right, top to bottom
#     sorted_data = sorted(extracted_data, key=lambda x: (x["center_y"], x["center_x"]))
#     merged_data = []
#     i = 0
#     while i < len(sorted_data):
#         current = sorted_data[i]
#         merged_text = current["text"]
#         merged_bbox = current["bbox"]
#         merged_confidence = [current["confidence"]]
#         merged_center_x = [current["center_x"]]
#         merged_center_y = [current["center_y"]]
        
#         j = i + 1
#         while j < len(sorted_data):
#             next_item = sorted_data[j]
            
#             # Check if the two regions are on the same line (within y_threshold)
#             y_diff = abs(next_item["center_y"] - current["center_y"])
#             if y_diff > y_threshold:
#                 break
            
#             # Check if the two regions are horizontally close (within x_threshold)
#             right_edge_current = current["bbox"][1][0]  # Right edge of current bbox
#             left_edge_next = next_item["bbox"][0][0]  # Left edge of next bbox
#             x_diff = left_edge_next - right_edge_current
#             if x_diff > x_threshold:
#                 break
            
#             # Merge the regions
#             merged_text += " " + next_item["text"]
#             # Create a new bounding box that encompasses both regions
#             merged_bbox = [
#                 [min(merged_bbox[0][0], next_item["bbox"][0][0]), min(merged_bbox[0][1], next_item["bbox"][0][1])],
#                 [max(merged_bbox[1][0], next_item["bbox"][1][0]), min(merged_bbox[1][1], next_item["bbox"][1][1])],
#                 [max(merged_bbox[2][0], next_item["bbox"][2][0]), max(merged_bbox[2][1], next_item["bbox"][2][1])],
#                 [min(merged_bbox[3][0], next_item["bbox"][3][0]), max(merged_bbox[3][1], next_item["bbox"][3][1])]
#             ]
#             merged_confidence.append(next_item["confidence"])
#             merged_center_x.append(next_item["center_x"])
#             merged_center_y.append(next_item["center_y"])
#             j += 1
        
#         # Calculate new center coordinates and average confidence
#         new_center_x = sum(merged_center_x) / len(merged_center_x)
#         new_center_y = sum(merged_center_y) / len(merged_center_y)
#         avg_confidence = sum(merged_confidence) / len(merged_confidence)
        
#         merged_data.append({
#             "text": merged_text,
#             "bbox": merged_bbox,
#             "center_x": new_center_x,
#             "center_y": new_center_y,
#             "confidence": avg_confidence
#         })
#         i = j if j > i + 1 else i + 1
    
#     return merged_data

# # Step 4: Calculate string similarity using Levenshtein distance
# def levenshtein_distance(s1, s2):
#     if len(s1) < len(s2):
#         return levenshtein_distance(s2, s1)
    
#     if len(s2) == 0:
#         return len(s1)
    
#     previous_row = range(len(s2) + 1)
#     for i, c1 in enumerate(s1):
#         current_row = [i + 1]
#         for j, c2 in enumerate(s2):
#             insertions = previous_row[j + 1] + 1
#             deletions = current_row[j] + 1
#             substitutions = previous_row[j] + (c1 != c2)
#             current_row.append(min(insertions, deletions, substitutions))
#         previous_row = current_row
    
#     return previous_row[-1]

# def string_similarity(s1, s2):
#     distance = levenshtein_distance(s1.lower(), s2.lower())
#     max_len = max(len(s1), len(s2))
#     if max_len == 0:
#         return 1.0
#     similarity = 1 - (distance / max_len)
#     return similarity

# # Step 5: Match predicted text with expected labels using similarity threshold
# def match_labels_with_similarity(extracted_data, fields, threshold=0.6):
#     label_positions = {}
#     for label, arabic_label in fields.items():
#         french_pos = None
#         arabic_pos = None
#         for item in extracted_data:
#             text = item["text"].strip()
            
#             # Compare with French label
#             if string_similarity(text, label) >= threshold:
#                 french_pos = (item["center_x"], item["center_y"], item["bbox"])
            
#             # Compare with Arabic label
#             if string_similarity(text, arabic_label) >= threshold:
#                 arabic_pos = (item["center_x"], item["center_y"], item["bbox"])
        
#         if french_pos and arabic_pos:
#             label_positions[label] = (french_pos, arabic_pos)
#         else:
#             print(f"Could not match both French and Arabic labels for '{label}'")
    
#     return label_positions

# # Step 6: Crop the region between French and Arabic labels for each field
# def crop_handwritten_regions(extracted_data, original_image):
#     # Define the French labels and their Arabic translations
#     fields = {
#         "Nom et prenom de l adherent": "إسم ولقب المنخرط",
#         "Numero CIN ou passeport": "عدد بطاقة لتعريف الوطنية او جواذ لسفر",
#         "Adresse de l'adherent": "عنوان المنخرط",
#         "Matricule CNAM": "رقم بطاقة الصندوق الوطني للتأمين على المرض",
#         "Matricule de l'adherent": "رقم المنخرط",
#         "Nom et prenom du malade": "اسم و لقب المريض",
#         "Date de naissance": "تاريخ الولادة"
#     }
    
#     # Create output folder
#     output_folder = "output_medical_form"
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)
    
#     # Match labels using similarity threshold
#     label_positions = match_labels_with_similarity(extracted_data, fields, threshold=0.7)
    
#     # Crop the region between French and Arabic labels for each field
#     for label, (french_pos, arabic_pos) in label_positions.items():
#         french_x, french_y, french_bbox = french_pos
#         arabic_x, arabic_y, arabic_bbox = arabic_pos
        
#         # Define the cropping region (between the French and Arabic labels)
#         x1 = int(french_bbox[1][0])  # Right edge of French label
#         x2 = int(arabic_bbox[0][0])  # Left edge of Arabic label
#         y1 = min(int(french_bbox[0][1]), int(arabic_bbox[0][1]))  # Top edge
#         y2 = max(int(french_bbox[2][1]), int(arabic_bbox[2][1]))  # Bottom edge

#         # Apply 20px vertical padding
#         height, width = original_image.shape[:2]
#         padding = 20
#         x1 = max(0, x1)
#         x2 = min(width, x2)
#         y1 = max(0, y1 - padding)
#         y2 = min(height, y2 + padding)

#         # Crop the region
#         if x2 > x1 and y2 > y1:
#             cropped_image = original_image[y1:y2, x1:x2]
            
#             # Save the cropped image
#             output_filename = label.lower().replace(" ", "_") + ".jpg"
#             output_path = os.path.join(output_folder, output_filename)
#             cv2.imwrite(output_path, cropped_image)
#             print(f"Saved cropped image for '{label}' to {output_path}")
#         else:
#             print(f"Invalid cropping region for '{label}'")


In [4]:
# image_path = "medical_form.jpg"
    
#     # Preprocess the image
# preprocessed_image, original_image = preprocess_image(image_path)



In [5]:
    
# # Extract text with EasyOCR
# extracted_data = extract_text_with_easyocr("preprocessed_form.jpg")


In [6]:
# merged_data = merge_adjacent_regions(extracted_data, y_threshold=5, x_threshold=5)


In [7]:
# print (extracted_data) 
# # Crop handwritten regions and save them
# crop_handwritten_regions(extracted_data, original_image)


## after fine-tunnig YOLOv8 model we are going to extract text using the bounding boxes predicted

## loading fine tuned model and creating the class names to be predicted as bounding box 

In [27]:
import cv2
import os
from ultralytics import YOLO

# Load the trained YOLOv8 model
model_path = "runs/detect/medical_form_yolo_best2/weights/best.pt"
model = YOLO(model_path)

# Define class names (must match the order in data.yaml)
class_names = [
    "nom et prenom de adherent",
    "matricule cnam",
    "matricule de adherent",
    "addresse de ladherent",
    "numero cin ou passeport",
    "nom et prenom du malade",
    "date de naissance",
    "date",
    "designation",
    "honoraire"
]



## predicting bounding boxes

In [28]:
# Create output folder for annotated images and cropped regions
output_folder = "predictions"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Path to the new image for prediction
image_path = "dataset/images/val/1432--5852526--20230720_page_0_1.jpg"  # Replace with the path to your new image
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"Could not load image at {image_path}")

# Run inference
results = model(image, imgsz=640, device=None)  # Use GPU (device=0); set to None for CPU



0: 640x480 1 nom et prenom de adherent, 1 matricule de adherent, 1 addresse de ladherent, 1 numero cin ou passeport, 1 nom et prenom du malade, 1 date de naissance, 2 dates, 1 designation, 2 honoraires, 151.6ms
Speed: 2.4ms preprocess, 151.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 480)


In [29]:
# Process the detected bounding boxes
crop_counter = 1
for result in results:
    boxes = result.boxes.xyxy  # Get bounding boxes in [x_min, y_min, x_max, y_max] format
    confidences = result.boxes.conf  # Get confidence scores
    class_ids = result.boxes.cls  # Get class IDs

    for box, conf, cls_id in zip(boxes, confidences, class_ids):
        # Extract bounding box coordinates
        x1, y1, x2, y2 = map(int, box)
        class_id = int(cls_id)
        label = class_names[class_id]
        confidence = float(conf)
        print (f"Confidence is : {confidence}")
        # # Apply 20px vertical padding for cropping
        # height, width = image.shape[:2]
        # padding = 0
        # x1 = max(0, x1)
        # x2 = min(width, x2)
        # y1 = max(0, y1 - padding)
        # y2 = min(height, y2 + padding)

        # Crop the detected region
        if x2 > x1 and y2 > y1:
            cropped_image = image[y1:y2, x1:x2]
            
            # Save the cropped image
            output_filename = f"{label.replace(' ', '_')}_{crop_counter}.jpg"
            output_path = os.path.join(output_folder, output_filename)
            cv2.imwrite(output_path, cropped_image)
            print(f"Saved cropped region to {output_path}")
            crop_counter += 1

# Save the annotated image
image_filename = os.path.basename(image_path)
annotated_image_path = os.path.join(output_folder, f"annotated_{image_filename}")
cv2.imwrite(annotated_image_path, image)
print(f"Saved annotated image to {annotated_image_path}")


Confidence is : 0.8790407180786133
Saved cropped region to predictions\date_1.jpg
Confidence is : 0.8396384119987488
Saved cropped region to predictions\designation_2.jpg
Confidence is : 0.8309372067451477
Saved cropped region to predictions\nom_et_prenom_de_adherent_3.jpg
Confidence is : 0.682129979133606
Saved cropped region to predictions\honoraire_4.jpg
Confidence is : 0.6554888486862183
Saved cropped region to predictions\date_de_naissance_5.jpg
Confidence is : 0.6499484777450562
Saved cropped region to predictions\nom_et_prenom_du_malade_6.jpg
Confidence is : 0.6109005808830261
Saved cropped region to predictions\addresse_de_ladherent_7.jpg
Confidence is : 0.608603835105896
Saved cropped region to predictions\honoraire_8.jpg
Confidence is : 0.5044302940368652
Saved cropped region to predictions\numero_cin_ou_passeport_9.jpg
Confidence is : 0.4248782694339752
Saved cropped region to predictions\matricule_de_adherent_10.jpg
Confidence is : 0.29880237579345703
Saved cropped region t

In [26]:
# Define paths
predictions_folder = "predictions"
output_json_path = os.path.join(predictions_folder, "extracted_text.json")

# Load the TrOCR model and processor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-handwritten")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set model to evaluation mode

# Dictionary to store extracted text
extracted_texts = {}

# Iterate over all images in the predictions folder
for image_name in os.listdir(predictions_folder):
    if not image_name.lower().endswith((".jpg", ".png", ".jpeg")):
        continue  # Skip non-image files (e.g., annotated images or JSON)

    image_path = os.path.join(predictions_folder, image_name)
    
    try:
        # Load the image
        image = Image.open(image_path).convert("RGB")
        
        # Preprocess the image
        pixel_values = processor(image, return_tensors="pt").pixel_values  # Shape: (1, 3, height, width)
        pixel_values = pixel_values.to(device)

        # Generate text
        with torch.no_grad():  # Disable gradient computation for inference
            generated_ids = model.generate(
                pixel_values,
                max_length=100,  # Adjust based on expected text length
                num_beams=4,    # Beam search for better results
                early_stopping=True
            )

        # Decode the generated IDs to text
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        
        # Store the extracted text
        extracted_texts[image_name] = generated_text
        print(f"Extracted text from {image_name}: {generated_text}")
        
    except Exception as e:
        print(f"Error processing {image_name}: {str(e)}")
        extracted_texts[image_name] = "Error: Could not extract text"

# Save the extracted texts to a JSON file
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(extracted_texts, f, ensure_ascii=False, indent=4)

print(f"Saved extracted texts to {output_json_path}")

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-large-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Extracted text from addresse_de_ladherent_7.jpg: ORBODY
Extracted text from annotated_1432--5852526--20230720_page_0_1.jpg: 0 0
Extracted text from date_1.jpg: 0 0
Extracted text from date_11.jpg: 0 0
Extracted text from date_de_naissance_5.jpg: 0 States
Extracted text from designation_2.jpg: 0 0
Extracted text from honoraire_4.jpg: 0 0
Extracted text from honoraire_8.jpg: 0 0
Extracted text from matricule_de_adherent_10.jpg: 0000386 " c
Extracted text from nom_et_prenom_de_adherent_3.jpg: Other ways.
Extracted text from nom_et_prenom_du_malade_6.jpg: Tabel Kim Anna
Extracted text from numero_cin_ou_passeport_9.jpg: 08368644.
Saved extracted texts to predictions\extracted_text.json
